In [1]:
import time as tm
import datetime as dt

# numpy
import numpy as np
# pandas
import pandas as pd

In [2]:
# Make pseudo data set:

timestamp_low = 1e9           # 2001-09-09
timestamp_high = 1e9 + 2e8    # 2008-01-11
total_unit = 200000
total_record = 1000000

codes = np.random.randint(low=0, 
                          high=total_unit, 
                          size=total_record)
lst_code = ["%8.8d"%(x) for x in codes] 

# lst_timestamp = np.random.randint(low=timestamp_low,
#                                   high=timestamp_high,
#                                   size=total_record)
x = np.random.random(total_record)
lst_timestamp = (timestamp_high - timestamp_low)*x + timestamp_low + x
lst_datetime = [str(dt.datetime.fromtimestamp(x)) for x in lst_timestamp]

dfData = pd.DataFrame(data=zip(lst_code, lst_datetime), columns=["Code", "DateTime"])
print dfData.head()
print dfData.tail()

       Code                    DateTime
0  00148681  2001-09-17 15:44:24.289153
1  00126608  2003-01-13 13:55:27.212562
2  00001220  2007-10-22 11:54:28.893134
3  00105798  2007-08-19 23:20:46.649255
4  00004619  2003-05-29 07:22:53.319421
            Code                    DateTime
999995  00185493  2006-11-02 22:10:07.857828
999996  00184652  2006-09-11 05:25:32.953270
999997  00086089  2005-09-20 09:45:01.212722
999998  00031302  2005-11-20 15:27:07.916446
999999  00174236  2006-09-07 12:31:04.216283


In [3]:
tk1 = tm.clock()

ts = dfData['DateTime'].apply(lambda x: dt.datetime.strptime(x,"%Y-%m-%d %H:%M:%S.%f"))  # datatime strint -> timestamp long int
dfData['TimeStamp'] = ts   # append timestamp as a new column of dfData

tk2 = tm.clock()
print dfData.head()
print dfData.tail()
print 'time counsumed = %1.3f(s)'%(tk2-tk1)

       Code                    DateTime                  TimeStamp
0  00148681  2001-09-17 15:44:24.289153 2001-09-17 15:44:24.289153
1  00126608  2003-01-13 13:55:27.212562 2003-01-13 13:55:27.212562
2  00001220  2007-10-22 11:54:28.893134 2007-10-22 11:54:28.893134
3  00105798  2007-08-19 23:20:46.649255 2007-08-19 23:20:46.649255
4  00004619  2003-05-29 07:22:53.319421 2003-05-29 07:22:53.319421
            Code                    DateTime                  TimeStamp
999995  00185493  2006-11-02 22:10:07.857828 2006-11-02 22:10:07.857828
999996  00184652  2006-09-11 05:25:32.953270 2006-09-11 05:25:32.953270
999997  00086089  2005-09-20 09:45:01.212722 2005-09-20 09:45:01.212722
999998  00031302  2005-11-20 15:27:07.916446 2005-11-20 15:27:07.916446
999999  00174236  2006-09-07 12:31:04.216283 2006-09-07 12:31:04.216283
time counsumed = 12.009(s)


In [4]:
tk3= tm.clock()

gpTS = dfData.groupby('Code', sort=True)['TimeStamp']   # Col'TimeStamp' is grouped by 'Code'
smax = gpTS.max()     # maximum timestamp value of each group
smin = gpTS.min()     # minimum timestamp value of each group
sdiff = smax - smin   # difference/time-span of each group

tk4= tm.clock()

print 'type=',type(sdiff)
print 'size=',len(sdiff)
print sdiff.head()

print 'time counsumed = %1.3f(s)'%(tk4-tk3)
print '\ntotal time consumed = %1.3f(s)'%((tk2-tk1)+(tk4-tk3))

type= <class 'pandas.core.series.Series'>
size= 198596
Code
00000000    159 days 18:15:41.550073
00000001             0 days 00:00:00
00000002   1323 days 05:38:06.906408
00000003    320 days 17:40:21.613439
00000004   2037 days 16:50:45.300261
Name: TimeStamp, dtype: timedelta64[ns]
time counsumed = 0.398(s)

total time consumed = 12.407(s)


In [5]:
# retrieve by code:

codex = '00000010'     # code = '00000010' 
diffx = sdiff[codex]   # time-delta = diffx
print 'type=',type(diffx)
print 'diff_ns=', diffx.value,'(ns)'  # time-delta value(numpy int64) in unit of ns
print 'diff_second=', diffx.value/1e9,'(s)'
print 'diff_day=', diffx.value/(1e9*3600*24),'(day)'

type= <class 'pandas._libs.tslib.Timedelta'>
diff_ns= 149519177798638000 (ns)
diff_second= 149519177.799 (s)
diff_day= 1730.54603934 (day)
